In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Transformação: converte para tensor e normaliza (média e desvio de MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Dataset de treino e teste
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Subset com os primeiros 1000 para treino e 500 para validação
train_subset = Subset(train_dataset, range(1000))
val_subset   = Subset(test_dataset, range(500))

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=64, shuffle=False)

In [ ]:
def show_batch(loader):
    images, labels = next(iter(loader))
    plt.figure(figsize=(10, 2))
    for i in range(10):
        img = images[i].squeeze().numpy() * 0.5 + 0.5  # desfaz a normalização
        plt.subplot(1, 10, i+1)
        plt.imshow(img, cmap='gray')
        plt.title(labels[i].item())
        plt.axis('off')
    plt.show()

show_batch(val_loader)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 6, 5),        # 28x28 → 24x24
            nn.ReLU(),
            nn.MaxPool2d(2),           # → 12x12
            nn.Conv2d(6, 12, 5),       # → 8x8
            nn.ReLU(),
            nn.MaxPool2d(2),            # → 4x4
            nn.Flatten(),                # Achata a saída para 1D
            nn.Linear(12 * 4 * 4, 64)   # Linear layer para reduzir a dimensionalidade
        )
        self.classifier = nn.Linear(64, 10)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

In [ ]:
model = SimpleCNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss:.4f} - Acc: {acc:.2f}%")

In [ ]:
model.eval()
embeddings = []
labels_list = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        x = model.encoder(images)
        embeddings.append(x.cpu())
        labels_list.append(labels)

X = torch.cat(embeddings).numpy()
y = torch.cat(labels_list).numpy()

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns


tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X)

plt.figure(figsize=(8, 8))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y, palette='tab10', s=30)
plt.title("t-SNE dos Embeddings da SimpleCNN")
plt.axis('off')
plt.show()

In [ ]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In [ ]:
# Escolher duas imagens do dataset
idx1, idx2 = 10, 13
img1, label1 = val_subset[idx1]
img2, label2 = val_subset[idx2]

# Visualizar as imagens
plt.figure(figsize=(4, 2))
plt.subplot(1, 2, 1)
plt.imshow(img1.squeeze() * 0.5 + 0.5, cmap='gray')
plt.title(f"Label: {label1}")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img2.squeeze() * 0.5 + 0.5, cmap='gray')
plt.title(f"Label: {label2}")
plt.axis('off')
plt.show()

# Extrair embeddings com encoder
model.eval()
with torch.no_grad():
    emb1 = model.encoder(img1.unsqueeze(0).to(device)).cpu().numpy()[0]
    emb2 = model.encoder(img2.unsqueeze(0).to(device)).cpu().numpy()[0]

# Calcular distância
dist = euclidean_distance(emb1, emb2)
print(f"Distância euclidiana entre os embeddings: {dist:.4f}")